# Tensorflow Feed Forward / Multi-Layer Perceptron Iris Dataset - Best Practices

Essentially, leverages estimators and layers API

In [119]:
# Load our data

import tensorflow as tf
import numpy as np
import pandas as pd

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

iris_data = load_iris()

IRIS_TRAINING = '../data/iris/iris_training.csv'
IRIS_TEST = '../data/iris/iris_test.csv'

training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

X_train = training_set.data
y_train = training_set.target
X_test = test_set.data
y_test = test_set.target

In [120]:
# Parameters
learning_rate = 0.1
num_steps = 10000
batch_size = 20
display_step = 10

# Network Parameters
n_hidden_1 = 20 # 1st layer number of neurons
n_hidden_2 = 20 # 2nd layer number of neurons
num_input = 4
num_classes = 3

In [121]:
# Convert numpy array to tensorflow estimator

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'features': X_train}, y=y_train,
    batch_size=batch_size, num_epochs=None, shuffle=True)

In [122]:
# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['features']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

In [123]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [124]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplkcub5d1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa689d1c1d0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [125]:
# Train the Model
model.train(input_fn, steps=num_steps)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



ValueError: Dimension 0 in both shapes must be equal, but are 20 and 1. Shapes are [20,3] and [1,20]. for 'softmax_cross_entropy_with_logits_sg' (op: 'SoftmaxCrossEntropyWithLogits') with input shapes: [20,3], [1,20].

In [ ]:
# Evaluate the Model
# Define the input function for evaluating
test_fn = tf.estimator.inputs.numpy_input_fn(
    x={'features': X_test}, y=y_test,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(test_fn)

In [ ]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'features': X_test}, shuffle=False)
# Use the model to predict the images class
preds = list(model.predict(input_fn))

for i, p in enumerate(preds):
    print("Pred:", p, "Actual:", y_test[i])